# Assignment 2 Q2

In [ ]:
# Imports
import numpy as np
import os
from utils.utilities import *
import time
from datetime import timedelta

## Prepare data

In [ ]:
X_train, y_train, list_ch_train = read_data(data_path="./data/Q2_dataset", split="train")
X_test, y_test, list_ch_test = read_data(data_path="./data/Q2_dataset", split="test")

In [ ]:
# Normalize
X_train, X_test = standardize(X_train, X_test)

In [ ]:
# One hot encoding the label
y_train = one_hot(y_train)
y_test = one_hot(y_test)

# Todo: Building 1D-CNN

In [ ]:
import tensorflow as tf

In [ ]:
series_length = 128
series_shape = (series_length)
num_channels = 9
num_classes = 6

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, series_length, num_channels], name='x')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [ ]:
def cnn(x):
    x_series = tf.reshape(x, [-1, series_length, num_channels])

    conv1 = tf.layers.conv1d(
        inputs=x_series,
        filters=16,
        kernel_size=[8],
        padding="same",
        activation=tf.nn.relu
    )

    pool1 = tf.layers.max_pooling1d(
        inputs=conv1, 
        pool_size=[8], 
        strides=8
    )

    conv2 = tf.layers.conv1d(
        inputs=pool1,
        filters=32,
        kernel_size=[4],
        padding="same",
        activation=tf.nn.relu
    )
    
    pool2 = tf.layers.max_pooling1d(
        inputs=conv2, 
        pool_size=[8], 
        strides=8
    )
    
    layer_shape = pool2.get_shape()
    num_features = layer_shape[1:3].num_elements()
    layer_flat = tf.reshape(pool2, [-1, num_features])
    
    fc1 = tf.layers.dense(
        inputs=layer_flat, 
        units=256, 
        activation=tf.nn.relu
    )

    logits = tf.layers.dense(
        inputs=fc1, 
        units=num_classes
    )

    return logits

In [ ]:
y_pred = cnn(x)

loss = tf.losses.mean_squared_error(
    labels=y_true,
    predictions=y_pred
)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

y_pred_cls = tf.argmax(y_pred, axis=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Todo: Run the network

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
total_iterations = 0
batch_size = 50

def optimize(num_epochs):
    for i in range(num_epochs):
        train_accuracy = []
        
        for x_batch, y_true_batch in get_batches(X_train, y_train, batch_size):
            feed_dict_train = {
                x: x_batch,
                y_true: y_true_batch
            }
            batch_accuracy, _ = sess.run([accuracy, optimizer], feed_dict=feed_dict_train)
            train_accuracy.append(batch_accuracy)
        
        print("Epoch {}, Train accuracy: {:.1%}".format(i, np.mean(train_accuracy)))
        if i % 5 == 0:
            print_test_accuracy()

def print_test_accuracy():
    test_accuracy = []
    
    for x_batch, y_true_batch in get_batches(X_test, y_test, batch_size):
        feed_dict_test = {
            x: x_batch,
            y_true: y_true_batch
        }
        batch_accuracy = sess.run(accuracy, feed_dict=feed_dict_test)
        test_accuracy.append(batch_accuracy)
    
    print("Test accuracy: {:.1%}".format(np.mean(test_accuracy)))

In [ ]:
print_test_accuracy()

In [ ]:
optimize(num_epochs=100)
print('='*30)
print_test_accuracy()